**https://www.youtube.com/watch?v=g6kQl_EFn84&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=7**

In [1]:
import torch
import torch.nn as nn   # All neural network modules nn.Linear, nn.conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all optimizations algorithms SGD, Adams etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset management and creates mini batches
import torchvision.datasets as datasets  # Has standard dataset we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset

**Saving Checkpoint and Model**

In [2]:
# Create CNN Module
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
  print("=> Saving checkpoint")
  torch.save(state, filename)

"""
def load_checkpoint(checkpoint):
  print("=> Loading checkpoint")
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['state_dict'])
"""

# Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5
#load_model = True

# Load Data
train_dataset = datasets.MNIST('/content/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST('/content/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize Network
model = CNN().to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#load_model(torch.load('/content/my_checkpoint.pth.tar'))

# Train Network
for epoch in range(num_epochs):
  losses = []
  
  if epoch == 2:
    checkpoint = {'state dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    save_checkpoint(checkpoint)

  for batch_idx,(data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)
    losses.append(loss.item())

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # Gradient Descent or Adam Step
    optimizer.step()
  
  mean_loss = sum(losses) / len(losses)
  print(f'Loss at epoch {epoch} is {mean_loss:.5f}')

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    #print(f'Got (num_correct)/(num_samples) with accuracy (float(num_correct)/float(num_samples)*100:.2f)')
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()

Loss at epoch 0 is 0.31987
Loss at epoch 1 is 0.10077
=> Saving checkpoint
Loss at epoch 2 is 0.07266
Loss at epoch 3 is 0.05895
Loss at epoch 4 is 0.05106


In [3]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59197/60000 with accuracy 98.66
Checking accuracy on test data
Got 9838/10000 with accuracy 98.38


**Load Checkpoint and Model**

In [6]:
# Create CNN Module
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
  print("=> Saving checkpoint")
  torch.save(state, filename)

def load_checkpoint(checkpoint):
  print("=> Loading checkpoint")
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['state_dict'])

# Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5
load_model = True

# Load Data
train_dataset = datasets.MNIST('/content/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST('/content/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize Network
model = CNN().to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

if load_model:
  load_checkpoint(load_model(torch.load('/content/my_checkpoint.pth.tar')))

# Train Network
for epoch in range(num_epochs):
  losses = []
  
  if epoch % 3 == 0:
    checkpoint = {'state dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    save_checkpoint(checkpoint)

  for batch_idx,(data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)
    losses.append(loss.item())

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # Gradient Descent or Adam Step
    optimizer.step()
  
  mean_loss = sum(losses) / len(losses)
  print(f'Loss at epoch {epoch} is {mean_loss:.5f}')

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    #print(f'Got (num_correct)/(num_samples) with accuracy (float(num_correct)/float(num_samples)*100:.2f)')
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()

TypeError: ignored

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)